In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#strona z listą wszystkich chmieli
site_hops = "https://twojbrowar.pl/pl/chmiele?id_category=18&n=133"

In [3]:
#budowa obiektów do analizy strony z listą chmieli
hop_site_resp = requests.get(site_hops)
hop_site_html = hop_site_resp.text
hop_site_soup = BeautifulSoup(hop_site_html,'html.parser')

In [4]:
# pętla budująca lonki dla chmieli na stornie
hop_link_start_string = "https://twojbrowar.pl/pl/surowce-piwowarskie/chmiele/" #po tym rozpoznajemy, że link dot. chmielu
hop_link_arr = [] #lista podstron z pojedynczym chmielem

for i in hop_site_soup.find_all("a", class_="product-name"):
    if hop_link_start_string in str(i["href"]):
        hop_link_arr.append(i["href"])

hop_link_arr = list(set(hop_link_arr))     # usuwaniu duplikatów obiekt set nie przyjmuje duplikatów a zamiana je usuwa   
#produktem tej pętli jest lista z linkami pojedynczych chmieli, każdy element listy jest inputem do kolejnej części procesu

In [5]:
#mechanizm pobierania informacji o każdym chmielu, wersja z danymi w formie DataFrame

indeks = []
atrybut = []
wartosc = []

for link in hop_link_arr: #iteracja przez każdą stronę z pojedynczym chmielem w celu pobrania informacji
    #konstrukcja obiektów do analizy strony
    odp = requests.get(link)
    html=odp.text
    soup = BeautifulSoup(html,'html.parser')
    
    #tabela z parametrami dla linku
    param_table = soup.find_all(class_='table-data-sheet')[0].find_all("tr")

    nazwa = soup.find_all("h1",itemprop="name")[0].text
  
    for row in param_table:
        #indeks
        indeks.append(nazwa)
        #atrybut
        atrybut.append(row.find_all("td")[1].text)
        #wartosc
        wartosc.append(row.find_all("td")[2].text)
    
    indeks.append(nazwa)
    atrybut.append("Dostępność")
    wartosc.append(soup.find_all(id='pb-available-title')[0].find("span").text)
    
    indeks.append(nazwa)
    atrybut.append("Cena")
    wartosc.append(soup.find_all(id='our_price_display')[0].text)


dane = {"Indeks":indeks,"Atrybut":atrybut,"Wartość":wartosc}

df_hops = pd.DataFrame(dane)
df_pivot_hops = df_hops.pivot_table(index='Indeks', columns='Atrybut', values='Wartość', aggfunc=lambda x: '; '.join(x))

In [6]:
df_pivot_hops.to_csv("C://Users//Rafał//Desktop//rafalcecotka-projekty//webscraping_twoj_browar//chmiel.csv")

In [7]:
df_pivot_hops.to_excel("C://Users//Rafał//Desktop//rafalcecotka-projekty//webscraping_twoj_browar//chmiel.xlsx")